## Informações sobre o aluno:

Nomes:

Matrícula:

## Questão 1

Você foi convidado para participar de um projeto de uma empresa BIA Technologies, uma empresa voltada para o comércio de jogos para video-games e computadores. Esse projeto conta com uma equipe de sistemas de recomendação, na qual você foi convidado a participar. Dessa forma, você irá auxiliar o desenvolvimento de um sistema de recomendação para essa empresa.

Considere o dataset fornecido pela empresa e faça uma análise básica desse dataset. Em seguida, utilize seus conhecimentos para desenvolver e avaliar três diferentes sistemas de recomendação da seguinte forma:

a) Desenvolva um sistema de recomendação utilizando filtragem colaborativa (sem realizar a redução da dimensionalidade dos dados);

b) Desenvolva um sistema de recomendação baseado em conteúdo (sem realizar a redução da dimensionalidade dos dados);

c) Desenvolva um sistema de recomendação utilizando a redução da dimensionalidade dos dados.

Forneça as métricas de avaliações para cada um dos sistemas. Conclua suas observações de forma a convencer o gestor do projeto sobre qual foi o melhor sistema de recomendação obtido até o momento.

**Entrega da resolução da prova: dia 03/12/2024 até as 23:59, via SIGAA.**

**Obs.:** A outra equipe de sistemas de recomendação da empresa BIA Technologies apresentou uma solução atingindo as métricas abaixo. Seu sistema deverá fornecer métricas melhores que as obtidas pela outra equipe (baseline):

'ndcg@5' = 0.0016818836315317604

'ndcg@10' = 0.003073280417540927

'precision@5' = 0.00016758439071103669

'precision@10' = 8.379219535551834e-05

## Conjunto de Dados

In [ ]:
import os
import json
import gzip
import pandas as pd

In [ ]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Video_Games_5.json.gz

--2024-11-23 21:54:50--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFilesSmall/Video_Games_5.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154050105 (147M) [application/x-gzip]
Saving to: ‘Video_Games_5.json.gz’

Video_Games_5.json. 100%[===================>] 146.91M   107MB/s    in 1.4s    

2024-11-23 21:54:52 (107 MB/s) - ‘Video_Games_5.json.gz’ saved [154050105/154050105]



In [ ]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Video_Games.json.gz

--2024-11-23 21:54:52--  https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/metaFiles2/meta_Video_Games.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53175983 (51M) [application/x-gzip]
Saving to: ‘meta_Video_Games.json.gz’

meta_Video_Games.js 100%[===================>]  50.71M  98.8MB/s    in 0.5s    

2024-11-23 21:54:52 (98.8 MB/s) - ‘meta_Video_Games.json.gz’ saved [53175983/53175983]



In [ ]:
def load_file(file_name):
    data = []
    with gzip.open(file_name) as f:
        for l in f:
            data.append(json.loads(l.strip()))

    print(file_name, len(data))
    df = pd.DataFrame.from_dict(data)
    return df

reviews_df = load_file("Video_Games_5.json.gz")
metadata_df = load_file("meta_Video_Games.json.gz")

Video_Games_5.json.gz 497577
meta_Video_Games.json.gz 84819


### Descrição das colunas
- reviewerID - ID do avaliador, por exemplo, A2SUAM1J3GNN3B
- asin - ID do produto, por exemplo, 0000013714
- reviewerName - nome do avaliador
- vote - votos úteis da avaliação
- style - um dicionário dos metadados do produto, por exemplo, "Formato" é "Capa Dura"
- reviewText - texto da avaliação
- overall - classificação do produto
- summary - resumo da avaliação
- unixReviewTime - tempo da avaliação (tempo Unix)
- reviewTime - tempo da avaliação (formato original)
- image - imagens que os usuários postam após receberem o produto

In [ ]:
reviews_df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5.0,True,"10 17, 2015",A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,NaN,NaN,NaN
1,4.0,False,"07 27, 2015",A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,NaN,NaN,NaN
2,3.0,True,"02 23, 2015",A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,NaN,NaN,NaN
3,2.0,True,"02 20, 2015",A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,NaN,NaN,NaN
4,5.0,True,"12 25, 2014",A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,NaN,NaN,NaN


### Descrição das colunas

- asin - ID do produto, por exemplo, 0000031852
- title - nome do produto
- feature - características do produto em formato de lista de itens
- description - descrição do produto
- price - preço em dólares americanos (no momento da coleta de dados)
- imageURL - URL da imagem do produto
- imageURLHighRes - URL da imagem do produto em alta resolução
- related - produtos relacionados (também comprados, também visualizados comprados juntos, comprar após visualizar)
- salesRank - informações do ranking de vendas
- brand - nome da marca
- categories - lista de categorias às quais o produto pertence
- tech1 - primeira tabela de detalhes técnicos do produto
- tech2 - segunda tabela de detalhes técnicos do produto
- similar - tabela de produtos similares

In [ ]:
metadata_df.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,,$0.72,0276425316,[],[],NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


Note que o dataset é muito grande e certas manipulações podem não caber na memória RAM. Além disso, alguns desses dados podem ser de baixa qualidade. Esses dados podem ser filtrados e você pode considerar apenas os dados de maior qualidade. Alguns filtros básicos:

In [ ]:
reviews_df_filtrado = reviews_df.copy()

# Conversão do unixReviewTime para um formato de data legível
reviews_df_filtrado['reviewDate'] = pd.to_datetime(reviews_df_filtrado['unixReviewTime'], unit='s')

# Removendo duplicatas e valores nulos
reviews_df_filtrado.dropna(subset=['reviewerID', 'asin', 'overall'], inplace=True)
reviews_df_filtrado.drop(['unixReviewTime', 'reviewTime'], axis=1, inplace=True)

In [ ]:
# Removendo duplicatas e valores nulos
metadata_df.drop_duplicates(subset=['asin'], inplace=True)
metadata_df.dropna(subset=['asin', 'title'], inplace=True)

In [ ]:
# Identificando os IDs de produtos únicos em metadata_df
produtos_validos = set(metadata_df['asin'])

# Filtrando reviews_df para manter apenas as linhas com produtos que estão em metadata_df
reviews_df_filtrado = reviews_df_filtrado[reviews_df_filtrado['asin'].isin(produtos_validos)]

# Verificando o resultado
print(f"Número original de linhas em reviews_df: {len(reviews_df)}")
print(f"Número de linhas após a filtragem: {len(reviews_df_filtrado)}")
reviews_df = reviews_df_filtrado

Número original de linhas em reviews_df: 497577
Número de linhas após a filtragem: 496315


In [ ]:
# Identificando os IDs de produtos únicos em reviews_df
produtos_validos = set(reviews_df['asin'])

# Filtrando metadata_df para manter apenas as linhas com produtos que estão em reviews_df
metadata_df_filtrado = metadata_df[metadata_df['asin'].isin(produtos_validos)]

# Verificando o resultado
print(f"Número original de linhas em metadata_df: {len(metadata_df)}")
print(f"Número de linhas após a filtragem: {len(metadata_df_filtrado)}")
metadata_df = metadata_df_filtrado

Número original de linhas em metadata_df: 71911
Número de linhas após a filtragem: 17389


## Resolução

- **Análise e preparação dos dados (0,5 pontos)**

In [ ]:
# Realize análise prévia dos dados. Prepare-os aplicando filtros e selecionando features relevantes.

# Insira comentários no decorrer do código

- **a) (2,0 pontos) Modelo de recomendação com filtragem colaborativa**

In [ ]:
# Implemente um sistema de recomendação utilizando filtragem colaborativa

# Insira comentários no decorrer do código

- **b) (2,0 pontos) Modelo de recomendação baseado em conteúdo**

In [ ]:
# Implemente um sistema de recomendação baseado em conteúdo

# Insira comentários no decorrer do código

- **c) (2,0 pontos) Modelo de recomendação aplicando a redução da dimensionalidade**

In [ ]:
# Implemente um sistema de recomendação aplicando a redução da dimensionalidade

# Insira comentários no decorrer do código

- **(2,0 pontos) Análise das métricas**

In [ ]:
# Calcule e informe as métricas para cada um dos modelos de recomendação desenvolvidos

# Utilize gráficos e tabelas sempre que considerar interessante

- **Conclusões (1,5 pontos)**

Insira suas conclusões aqui.

Dica: Informe as conclusões obtidas. Compare os valores das métricas obtidas entre os modelos e justifique qual seria o melhor modelo a ser utilizado no momento. Foque em convencer o gestor de que os seus modelos de recomendações são bons e qual seria o melhor a ser utilizado apresentando o motivo/justificativa.